<a href="https://colab.research.google.com/github/vnewsome/project_voting/blob/main/ML_project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Summary

Data

In [3]:
! pip3 install mapclassify
# don't need to run again

In [1]:
! git clone https://github.com/vnewsome/project_voting

Cloning into 'project_voting'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 79 (delta 8), reused 5 (delta 5), pack-reused 68
Receiving objects: 100% (79/79), 31.37 MiB | 11.33 MiB/s, done.
Resolving deltas: 100% (33/33), done.
Updating files: 100% (42/42), done.


In [4]:
# packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mapclassify as mp

In [6]:
# data loading
voting = pd.read_csv('/content/project_voting/data/voting_VA.csv')
adj = pd.read_csv('/content/project_voting/data/county_adjacencies.csv')
# there are about 20 different datasets in county data, can load more later
# should probably look at income, income inequality, education, race

In [7]:
voting.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL


In [8]:
voting.describe()

,Unnamed: 0,year,county_fips,candidatevotes,totalvotes,version
count,3736.000000,3736.000000,3736.000000,3736.000000,3736.000000,3736.0
mean,47061.027837,2012.826552,51264.579229,5878.282120,29116.353587,20220315.0
std,23219.785408,7.589036,270.546458,18452.435802,58562.057257,0.0
min,11161.000000,2000.000000,51001.000000,0.000000,0.000000,20220315.0
25%,21235.750000,2004.000000,51069.000000,59.000000,6741.500000,20220315.0
50%,49016.500000,2016.000000,51139.000000,899.000000,11629.000000,20220315.0
75%,71036.250000,2020.000000,51550.000000,4548.250000,24520.750000,20220315.0
max,71970.000000,2020.000000,51840.000000,355133.000000,601243.000000,20220315.0


In [ ]:
# From the VA Dept of Emergency Management, get the shapefiles: https://vgin.vdem.virginia.gov/datasets/777890ecdb634d18a02eec604db522c6/about
# From that archive, I uploaded the following files to Colab:
# VirginiaCounty_ClippedToShoreline.cpg
# VirginiaCounty_ClippedToShoreline.dbf
# VirginiaCounty_ClippedToShoreline.prj
# VirginiaCounty_ClippedToShoreline.shp
# VirginiaCounty_ClippedToShoreline.shp.xml
# VirginiaCounty_ClippedToShoreline.shx

# The IPUMS data is in the github repo. What are these data? IPUMS is a microdata service that mostly distributes versions of the Census data and the American Community Survey, but also does a lot of international data. I downloaded a bunch of county level (NHGIS) data that looked interesting or promising. A lot of the files have similar data in each file, but they all have codebooks. You probably want to flip through a few and think about which have interesting covariates for predicting R vs D voting (e.g. education and education*race, income and income*race, etc.).

# There is a link to FRED for macro data (inflation, unemployment, etc.) in the readme. You can build a model and then plug in projections for November. That's another interesting angle to follow up, if you're interested: Uncertainty driven by the uncertainty in macro projections for the next 6 months.

Results

Conclusion

Appendix